In [ ]:
!pip install --upgrade --user google-cloud-aiplatform>=1.29.0 google-cloud-storage

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
google-colab 1.0.0 requires google-auth==2.17.3, but you have google-auth 2.26.1 which is incompatible.


In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
# get project ID
PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"
if PROJECT_ID == "(unset)":
    print(f"Please set the project ID manually below")

In [ ]:
# define project information
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]"

# generate a unique id for this session
from datetime import datetime
UID = datetime.now().strftime("%m%d%H%M")
print(PROJECT_ID)

vertext-ai-dar


In [ ]:
! gcloud services enable compute.googleapis.com aiplatform.googleapis.com storage.googleapis.com --project {PROJECT_ID}

Operation "operations/acat.p2-982785856251-ba27b0dd-eb28-472d-a1a3-47a89a927167" finished successfully.


In [ ]:
BUCKET_URI = f"gs://{PROJECT_ID}-vs-quickstart-{UID}"

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI
! gsutil cp "gs://github-repo/data/vs-quickstart/product-embs.json" $BUCKET_URI

Creating gs://vertext-ai-dar-vs-quickstart-01071516/...
Copying gs://github-repo/data/vs-quickstart/product-embs.json [Content-Type=application/json]...
- [1 files][ 79.3 MiB/ 79.3 MiB]                                                
Operation completed over 1 objects/79.3 MiB.                                     


In [ ]:
! gsutil cp "gs://github-repo/data/vs-quickstart/product-embs.json" . # for query tests

Copying gs://github-repo/data/vs-quickstart/product-embs.json...
\
Operation completed over 1 objects/79.3 MiB.                                     


In [ ]:
! head product-embs.json


{"id":"11863","name":"wacoal women's embrace lace chemise - 814191","embedding":[0.024530868977308273,-0.040793128311634064,0.00047576811630278826,0.017147192731499672,0.026629658415913582,-0.06141049787402153,0.020689481869339943,0.032609399408102036,-0.043242260813713074,-0.0030736858025193214,0.029859514907002449,-0.0070092626847326756,0.0074279718101024628,0.0044401995837688446,0.012789646163582802,0.0056940736249089241,-0.050746820867061615,-0.016834203153848648,-0.0025733353104442358,0.030782531946897507,-0.028840042650699615,0.013945236802101135,-0.0078231878578662872,-0.073326528072357178,-0.028099022805690765,-0.091063208878040314,0.016907420009374619,-0.0062174019403755665,0.0086391577497124672,-0.021324845030903816,-0.030394084751605988,-0.042661987245082855,-0.021932870149612427,-0.024862896651029587,0.025864196941256523,0.0095110638067126274,-0.023645631968975067,0.018030215054750443,0.013453914783895016,0.014843377284705639,-0.021920621395111084,0.0013795378617942333,-0.0

In [ ]:
# init the aiplatform package
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
# create Index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name = f"vs-quickstart-index-{UID}",
    contents_delta_uri = BUCKET_URI,
    dimensions = 768,
    approximate_neighbors_count = 10,
)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Creating MatchingEngineIndex
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Create MatchingEngineIndex backing LRO: projects/982785856251/locations/us-central1/indexes/6579798638006894592/operations/7457699230438653952
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex created. Resource name: projects/982785856251/locations/us-central1/indexes/6579798638006894592
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:To use this MatchingEngineIndex in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:index = aiplatform.MatchingEngineIndex('projects/982785856251/locations/us-central1/indexes/6579798638006894592')


In [ ]:
## create `IndexEndpoint`
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name = f"vs-quickstart-index-endpoint-{UID}",
    public_endpoint_enabled = True
)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Creating MatchingEngineIndexEndpoint
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Create MatchingEngineIndexEndpoint backing LRO: projects/982785856251/locations/us-central1/indexEndpoints/2513048174491336704/operations/7876956208249176064
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint created. Resource name: projects/982785856251/locations/us-central1/indexEndpoints/2513048174491336704
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:To use this MatchingEngineIndexEndpoint in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/982785856251/locations/us-central1/indexEndpoints/2513048174491336704')


In [ ]:
DEPLOYED_INDEX_ID = f"vs_quickstart_deployed_{UID}"
print(DEPLOYED_INDEX_ID)

vs_quickstart_deployed_01071516


In [ ]:
# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(
    index = my_index, deployed_index_id = DEPLOYED_INDEX_ID
)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/982785856251/locations/us-central1/indexEndpoints/2513048174491336704
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/982785856251/locations/us-central1/indexEndpoints/2513048174491336704/operations/849088989737517056
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/982785856251/locations/us-central1/indexEndpoints/2513048174491336704


resource name: projects/982785856251/locations/us-central1/indexEndpoints/2513048174491336704

In [ ]:
import json

# build dicts for product names and embs
product_names = {}
product_embs = {}
with open('product-embs.json') as f:
    for l in f.readlines():
        p = json.loads(l)
        id = p['id']
        product_names[id] = p['name']
        product_embs[id] = p['embedding']

In [ ]:
 # Get the embedding for ID 6523 "cloudveil women's excursion short" you can also try with other IDs such as 12711, 18090, 19536 and 11863
query_emb = product_embs['6523']

print(query_emb)

[-0.015140533447265625, 0.029022620990872383, 0.043999187648296356, 0.0008045680006034672, 0.02479265257716179, -0.058345310389995575, 0.010426630266010761, 0.023504989221692085, -0.03466186299920082, -0.00134370313026011, 0.007397875655442476, -0.01431096438318491, 0.024990102276206017, 0.06665688753128052, 0.023334601894021034, -0.005286165047436953, -0.06492510437965393, -0.0345313623547554, 0.060259561985731125, 0.010223621502518654, -0.09199754148721695, 0.01886577345430851, 0.03483972325921059, -0.027113549411296844, -0.03256196156144142, -0.07872982323169708, 0.037879571318626404, -0.009713241830468178, -0.03232517093420029, -0.07063174992799759, 0.0024606185033917427, -0.015956062823534012, -0.003946097567677498, 0.021167505532503128, -0.008327499032020569, 0.055032506585121155, 0.019084438681602478, 0.0015176940942183137, 0.00926684495061636, 0.06493163108825684, 0.0036904136650264263, 0.02693367190659046, 0.04891353100538254, -0.001483380445279181, -0.0366176962852478, -0.013

In [ ]:
# run query
response = my_index_endpoint.find_neighbors(
    deployed_index_id = DEPLOYED_INDEX_ID,
    queries = [query_emb],
    num_neighbors = 10
)

# show the results
for idx, neighbor in enumerate(response[0]):
    print(f"{neighbor.distance:.2f} {neighbor.id} {product_names[neighbor.id]}")

1.00 6523 cloudveil women's excursion short
0.82 6669 quiksilver womens cruiser short
0.80 6467 xcvi women's alisal short
0.80 28280 cloudveil men's kahuna short
0.78 6655 ibex women's gozo short
0.78 6304 sanctuary clothing women's coquette short
0.78 6739 sunner women's collins printed short
0.77 6804 hurley lowrider cargo 2.5 short - women's
0.77 6770 stitch's women's fox knee length short
0.77 7211 sanctuary clothing women's passenger skirt


# SEARCH the Index

In [ ]:
from vertexai.language_models import TextEmbeddingModel


def text_embedding(text_to_embed) -> list:
    """Text embedding with a Large Language Model."""
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
    embeddings = model.get_embeddings([text_to_embed])
    for embedding in embeddings:
        vector = embedding.values
        print(f"Length of Embedding Vector: {len(vector)}")
    return vector


## Specify a search

In [ ]:
SEARCH_TERM = "Winter Jacket with Hood for men"

In [ ]:
SEARCH_TERM_EMBEDDING = text_embedding(SEARCH_TERM)

print(SEARCH_TERM_EMBEDDING)

Length of Embedding Vector: 768
[0.006585866678506136, -0.0031216039787977934, 0.05027128383517265, 0.022746017202734947, 0.0608673058450222, -0.006805539131164551, -0.016818484291434288, 0.0896369144320488, 0.019613003358244896, -0.01013384759426117, -0.03428211808204651, 0.009057072922587395, 0.019853737205266953, -0.01222735084593296, -0.006660302635282278, -0.019326023757457733, -0.06727059930562973, -0.03143735229969025, 0.057832103222608566, 0.026862086728215218, -0.061578184366226196, 0.05491599068045616, 0.034154877066612244, -0.05177997797727585, 0.005032537039369345, -0.06479024887084961, 0.011419030837714672, -0.02082798443734646, -0.0004747669445350766, -0.000299947801977396, -0.01880752295255661, -0.008261716924607754, -0.021162306889891624, 0.04735545068979263, -0.04595936834812164, 0.03436174988746643, -0.019500259310007095, -0.019494349136948586, 0.02819647081196308, 0.035420503467321396, 0.029203146696090698, 0.0025679338723421097, 0.02597673237323761, 0.02795865572988

## Query the index using the embedding created from the search term

In [ ]:
response = my_index_endpoint.find_neighbors(
    deployed_index_id = DEPLOYED_INDEX_ID,
    queries = [SEARCH_TERM_EMBEDDING],
    num_neighbors = 10
)

# show the results
for idx, neighbor in enumerate(response[0]):
    print(f"{neighbor.distance:.2f} {neighbor.id} {product_names[neighbor.id]}")

0.78 24256 mammut men's ultimate hoody jacket
0.78 24358 rvca men's puffer fleece jacket
0.77 17687 tri-mountain mens nylon hooded jacket with fleece lining.
0.76 17685 tapout men's tri fecta hooded jacket
0.76 24295 chestnut hill men's polartec full zip fleece jacket. ch950
0.76 18224 puma men's fleece hooded track jacket
0.76 24381 dockers men's plush open bottom jacket with scarf
0.76 20333 calvin klein sportswear men's double flap 4 pocket fur trim jacket
0.75 24158 charles river apparel men's windproof hooded rain jacket. 9199
0.75 17263 neff men's fuzzy hood
